https://ariepratama.github.io/Introduction-to-BLEU-in-python/

https://towardsdatascience.com/bleu-bilingual-evaluation-understudy-2b4eab9bcfd1

In [1]:
import warnings
warnings.filterwarnings('ignore')

import nltk.translate.bleu_score as bleu

In [2]:
# split will transform the sentence into 1-gram
reference = [
    'this is a ship'.split(),
    'it is ship'.split(),
    'ship it is'.split(),
    'a ship, it is'.split() # master Yoda
]

In [3]:
translation = 'it is ship'.split()

In [4]:
print('BLEU score: {}'.format(bleu.sentence_bleu(reference, translation)))

BLEU score: 1.2213386697554703e-77


In [5]:
import numpy as np
from nltk import ngrams
from collections import Counter

In [6]:
def count_ngram(unigram, ngram=1):
    """
    Return
    -----
    counter: dict, containing ngram as key, and count as value
    """
    return Counter(ngrams(unigram, ngram))

In [7]:
test_unigram = 'this is is a ship'.split()
res = count_ngram(test_unigram)

assert res[('is',)] == 2
assert res[('a',)] == 1

print('res: {}'.format(res))

res: Counter({('is',): 2, ('this',): 1, ('a',): 1, ('ship',): 1})


In [8]:
res2 = count_ngram('abc is abc is abc'.split() , 2)

In [9]:
'abc is abc is abc'.split()

['abc', 'is', 'abc', 'is', 'abc']

In [10]:
res2

Counter({('abc', 'is'): 2, ('is', 'abc'): 2})

In [11]:
from IPython.core.debugger import Pdb;

In [12]:
translation = 'is some'
references = [
    'this is a test',
    'this is is is test'
]

In [13]:
def count_clip_ngram(translation_u, list_of_reference_u, ngram=1):
    """
    Return
    ----
    
    """
    res = dict()
    # retrieve hypothesis counts
    ct_translation_u = count_ngram(translation_u, ngram=ngram)
    
    # retrieve translation candidate counts
    for reference_u in list_of_reference_u:
        ct_reference_u = count_ngram(reference_u, ngram=ngram)
        #Pdb().set_trace()
        for k in ct_reference_u:
            if k in res:
                res[k] = max(ct_reference_u[k], res[k])
            else:
                res[k] = ct_reference_u[k]
                
    #Pdb().set_trace()
    return {
        k: min(ct_translation_u.get(k, 0), res.get(k, 0)) 
        for k in ct_translation_u
    }

In [14]:
res = count_clip_ngram(
    translation.split(), 
    list(map(lambda ref: ref.split(), references))
)


assert res[('is',)] == 1
assert res[('some',)] == 0

print('result from count clip: {}'.format(res))

result from count clip: {('is',): 1, ('some',): 0}


```python
ipdb>  ct_translation_u
Counter({('is',): 1, ('some',): 1})

ipdb>  res
{('this',): 1, ('is',): 3, ('a',): 1, ('test',): 1}
```

In [15]:
def modified_precision(translation_u, list_of_reference_u, ngram=1):
    ct_clip = count_clip_ngram(translation_u, list_of_reference_u, ngram)
    ct = count_ngram(translation_u, ngram)
    
    return sum(ct_clip.values()) / float(max(sum(ct.values()), 1))